imports

In [1]:
import pandas as pd
import numpy as np

import qcew_2010_2021
# import all functions from qcew module
from qcew_2010_2021 import *

import msa_codes
from msa_codes import *


step 1: download all data from online BLS database

In [2]:
df = extract_2010()

KeyboardInterrupt: 

step 2: add geographic tag to each row

In [ ]:
df_geo = get_2014_industry_with_bls_geographic_tag(df)

step 3: create total employment dataframe

In [ ]:
#make total df of 2010-2013 data
total_employment = df_geo.query("industry_code == '10' & own_code == 0")
#rename employee column as total for future merge
total_employment = total_employment.rename(columns={'annual_avg_emplvl':'total_employees'})

#keep columns that are in run_qcew.ipynb
total_employment_df = total_employment[['area_fips','area_title','geographic_level','year','total_employees']]

step 4: create fossil employment dataframe

In [ ]:
#make fossil employment

FOSSIL_NAICS_CODES = ['2121','211','213','23712','486','4247','22112']


fossil_employment = df_geo.loc[df_geo['industry_code'].isin(FOSSIL_NAICS_CODES)]


# group by all ownership codes

total_fossil_employment = fossil_employment.groupby(['area_fips','area_title','geographic_level','year']).agg({'annual_avg_emplvl':'sum'}).reset_index()

total_fossil_employment = total_fossil_employment.rename(columns={'annual_avg_emplvl':'fossil_employees'})

total_fossil_employment_short = total_fossil_employment[['area_fips','area_title','geographic_level','year','fossil_employees']]

step 5: combine total and fossil employment data

In [ ]:
#combine for total and fossil 2010-2013

combined = total_employment_df.merge(total_fossil_employment_short,on=['area_fips','area_title','geographic_level','year'],how='left')
combined['year'] = combined['year'].astype(str)

step 6: cleaning of geographic identifier columns

In [ ]:
#geoid processing

# clean area fips column first
combined['area_fips'] = combined['area_fips'].astype(str)
#make 5 digits for 4 digit occurences
combined['area_fips'] = combined['area_fips'].apply(lambda x:str(x).zfill(5))
#make separate column for MSA logic
combined['geoid'] = combined['area_fips']
# take out C in MSA to add extra 0 
combined['geoid'] = combined['geoid'].str.replace('C','')
# for MSAs, make geoid to match census crosswalk
combined['geoid'] = np.where(combined['geographic_level']=='metropolitan_stat_area',combined['geoid'] + '0',combined['geoid'])
# only keep geo levels we need (e.g. remove state and country totals)
final_employment = combined.query("geographic_level=='county' or geographic_level=='metropolitan_stat_area'")

In [ ]:
msa = msa_codes.extract()

msa['geoid'] = msa['geoid'].astype(str)
employment = final_employment.merge(msa,on='geoid',how='inner')
employment['fips_county'] = np.where(employment['geographic_level']=='county',employment['geoid'],employment['FIPS code'] + employment['County code'])

step 7: create eligibility criteria # 1) % fossil employees

In [ ]:
employment['percent_fossil_employment'] = employment['fossil_employees'] / employment['total_employees'] * 100
employment['meets_fossil_threshold'] = np.where(employment['percent_fossil_employment'] > .17,1,0)

step 8: create eligibility criteria # 2) unemployment rate higher than previous years

In [ ]:
# read in unemployment file
unemployment = pd.read_csv('/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/notebooks/files/unemployment.csv',dtype={'geoid':'str','year':'str','state_code':'str'})

# make column a float for comparison
unemployment['local_area_unemployment'] = pd.to_numeric(unemployment['local_area_unemployment'],errors='coerce')

#fix geoid for merge with employment

unemployment['geoid'] = np.where(unemployment['geographic_level']=='county',unemployment['state_code'] + unemployment['geoid'],unemployment['geoid'].str[0:5])

employment_both = employment.merge(unemployment,on=['geoid','geographic_level','year'],how='inner')
employment_both['meets_unemployment_criteria'] = np.where(employment_both['local_area_unemployment'] > employment_both['national_unemployment_rate_prev_year'],1,0)

step 9: identify which areas meet the unemployment criteria

In [ ]:
eligible_employment_areas = employment_both.query("meets_fossil_threshold==1 & meets_unemployment_criteria==1")

# clean for export
eligible_employment_areas = eligible_employment_areas.rename(columns={'May 2021 MSA name':'msa_name'})
eligible_employment_areas = eligible_employment_areas[['area_title','geographic_level','State','msa_name','geoid','percent_fossil_employees','meets_fossil_threshold','local_area_unemployment','national_unemployment_rate_prev_year','meets_unemployment_criteria','fips_county']]

step 10: export with all data

In [ ]:
eligible_employment_areas.to_csv('/Users/mcastillo/Documents/Github/rmi-energy-communities/notebooks/files/qualifying_employment_areas.csv')

step 11: narrower export for patio

In [ ]:
patio_employment = eligible_employment_areas[['area_title','fips_county']]

# columns in patio
patio_employment['qualifying_area'] = 'msa_or_county'

patio_employment['criteria'] = 'fossil_employment'

In [ ]:
patio_employment.to_csv('/Users/mcastillo/Documents/Github/rmi-energy-communities/notebooks/files/patio_employment.csv')